In [50]:
%matplotlib inline
%pwd

'/home/gabe/work/fast-ai/nbs'

In [119]:
%%html
<style>
  .end_space {
      min-height: 1000px;
  }
</style

In [99]:
DATA_DIR = '../data/statefarm'
BATCH_SIZE = 64

In [100]:
import sys
import os
from importlib import reload

sys.path.append('../src')

In [178]:
import utils.statefarm
reload(utils.statefarm)

import utils.trainhelper
reload(utils.trainhelper)

from utils.statefarm import *
from utils.trainhelper import get_batches

In [104]:
t_batches = get_batches(os.path.join(DATA_DIR, 'train'))

print(t_batches.filenames[:5], 'filenames')
print(t_batches.num_class, 'classes')
print(t_batches.image_shape, 'image shape')

Found 22424 images belonging to 10 classes.
['c0/img_82409.jpg', 'c0/img_95245.jpg', 'c0/img_88538.jpg', 'c0/img_45571.jpg', 'c0/img_14492.jpg'] filenames
10 classes
(224, 224, 3) image shape
